In [1]:
import requests

# Specify the URL of the page you want to download
url = "https://pokemondb.net/pokedex/all"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to an HTML file
    with open("index.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Page saved as 'index.html'")
else:
    print(f"Failed to download page. Status code: {response.status_code}")

Page saved as 'index.html'


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# If the HTML is saved locally, you can use:
with open("index.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Create Pokedex-List
pokedex = []

# Filter HTML-file for table data
pokemon = soup.select("tbody > tr")

# Cycle through all Pokemon
for p in pokemon:

    # Parse for every value
    id = p.select_one("td.cell-fixed > span").text
    name = p.select_one("td.cell-name > a").text
    type = [i.text for i in p.select("td.cell-icon > a")]
    total = p.select_one("td.cell-total").text
    hp, attack, defense, sp_atk, sp_def, speed = [i.text for i in p.select("td[class='cell-num']")]

    # Add Pokemon to Pokedex
    pokedex.append({"id": id,
                    "name": name,
                    "type": type,
                    "total": total,
                    "hp": hp,
                    "attack": attack,
                    "defense": defense,
                    "sp_atk": sp_atk,
                    "sp_def": sp_def,
                    "speed": speed})

# Convert to Dataframe
df = pd.DataFrame(pokedex)
print("Successfully collected all",len(df),"Pokemon to Dataframe.")

Successfully collected all 1215 Pokemon to Dataframe.


In [26]:
# Exploratory Analysis using pandaSQL-Library
from pandasql import sqldf

# Explode type-Column to make Dataframe SQL-Compatible
df_ex = df.explode("type")

# 2a. Strongest Pokemon of each type
print(sqldf("""
SELECT id,name,type,max(total) as max_total
FROM df_ex
GROUP BY type
ORDER BY max_total DESC"""))
print()

# 2b. Strongest Attackers
print(sqldf("""
SELECT *
FROM df_ex
ORDER BY attack DESC
LIMIT 5"""))
print()

# 2c. Average Stats for each type
print(sqldf("""
SELECT type, avg(total), avg(hp), avg(attack), avg(defense), avg(sp_atk), avg(sp_def), avg(speed)
FROM df_ex
GROUP BY type
ORDER BY type"""))

      id       name      type max_total
0   0150     Mewtwo   Psychic       780
1   0384   Rayquaza    Flying       780
2   0150     Mewtwo  Fighting       780
3   0384   Rayquaza    Dragon       780
4   0382     Kyogre     Water       770
5   0383    Groudon    Ground       770
6   0383    Groudon      Fire       770
7   0493     Arceus    Normal       720
8   0376  Metagross     Steel       700
9   0248  Tyranitar      Rock       700
10  0646     Kyurem       Ice       700
11  0719    Diancie     Fairy       700
12  0248  Tyranitar      Dark       700
13  0890  Eternatus    Poison       690
14  0487   Giratina     Ghost       680
15  0644     Zekrom  Electric       680
16  0254   Sceptile     Grass       630
17  0127     Pinsir       Bug       600

     id        name      type total  hp attack defense sp_atk sp_def speed
0  0512    Simisage     Grass   498  75     98      63     98     63   101
1  0514    Simisear      Fire   498  75     98      63     98     63   101
2  0516    Sim